# The Viewer

In [ ]:
import numpy as np
import json
import time
import ipywidgets as widgets
from cad_viewer_widget import (
    AnimationTrack, CadViewer, show, open_viewer, 
    get_sidecar, get_sidecars, close_sidecar, close_sidecars, get_default_sidecar, set_default_sidecar
)

from cad_viewer_widget.utils import numpyify

names = ["hexapod", "box", "box1", "boxes", "faces", "edges", "vertices", "box-faces", "box-edges", "box-vertices", "longbox"]
objects = {}
states = {}

for name in names:
    with open(f"../examples/{name}.json", "r") as fd:
        objects[name] = numpyify(json.load(fd))

    with open(f"../examples/{name}-states.json", "r") as fd:
        states[name] = json.load(fd)

## Cell view

In [ ]:
name = "boxes"
control = "trackball"
#control = "orbit"

cv = show(
    objects[name], 
    states[name],
    control=control,
    cad_width=700,
    height=525,
    glass=False,
    js_debug=True,
    collapse=1,
    theme="browser",
)

## Exports

**Modify view before exporting**

In [ ]:
cv.export_png("boxes.png")

In [ ]:
cv.export_html()

**Pin as PNG**

In [ ]:
cv.pin_as_png() # same as pressing the pin top right button

# Sidecar handling

## openviewer and add_shapes

In [ ]:
cv1 = open_viewer(
    title="CVW 1",
    anchor="split-right",
    cad_width=700,
    tree_width=250,
    height=525,
    glass=True
)

In [ ]:
name = "hexapod"
cv1.add_shapes(
    objects[name], 
    states[name],
    #tools=False,
    #ortho=False,
    control="trackball",
    axes=False,
    axes0=False,
    grid=(True,True, False),
    ticks=10,
    transparent=True,
    #black_edges=True,

    normal_len=0,
    default_edge_color="#707070",
    default_opacity=0.5,
    ambient_intensity=0.5,
    direct_intensity=0.3,

    reset_camera=True,
    position = (865.4844022079983, -276.23389988421786, 335.21716816984906),
    quaternion = (0.43557639340677845, 0.3648618806188253, 0.4409953598863984, 0.6947460731351442),
    zoom=0.8,
    timeit=False,

    zoom_speed=0.5,
    pan_speed=0.5,
    rotate_speed=1.0,
    js_debug=True,
)

## Show command

In [ ]:
name = "boxes"
cv2 = show(
    objects[name], 
    states[name],
    title="CVW 2",
    anchor="split-right",
    height=600,
    cad_width=800,
    
    ortho=False,
    control="orbit",
    axes=True,
    grid=(True, False, False),
    ticks=40,
    transparent=True,
    #black_edges=True,

    normal_len=2,
    default_edge_color="#707070",
    default_opacity=0.5,
    ambient_intensity=0.5,
    direct_intensity=0.3,   
)

In [ ]:
cv1.close()

In [ ]:
get_sidecars()

In [ ]:
name = "hexapod"
cv = show(
    objects[name], 
    states[name],
    title="CVW 2",
    collapse=1,
    glass=True,
)

In [ ]:
import numpy as np

from cad_viewer_widget import AnimationTrack

horizontal_angle = 25

def intervals(count):
    r = [ min(180, (90 + i*(360 // count)) % 360) for i in range(count)]
    return r 

def times(end, count):
    return np.linspace(0, end, count+1)
    
def vertical(count, end, offset, reverse):
    ints = intervals(count)
    heights = [round(35 * np.sin(np.deg2rad(x)) - 15, 1) for x in ints]
    heights.append(heights[0])
    return times(end, count), heights[offset:] + heights[1:offset+1]

def horizontal(end, reverse):
    factor = 1 if reverse else -1
    return times(end, 4), [0, factor * horizontal_angle, 0, -factor * horizontal_angle, 0]


leg_group = ("left_front", "right_middle", "left_back")
leg_names = ['right_back', 'right_middle', 'right_front', 'left_back', 'left_middle', 'left_front']

for name in leg_names:
    # move upper leg
    cv.add_track(AnimationTrack(
        f"/bottom/{name}", 
        "rz", *horizontal(4, "middle" in name)
    ))

    # move lower leg
    cv.add_track(AnimationTrack(
        f"/bottom/{name}/lower", 
        "rz", *vertical(8, 4, 0 if name in leg_group else 4, "left" in name)
    ))

cv.animate(3)

In [ ]:
cv2.close()
cv2.disposed

In [ ]:
get_sidecars()

## Use default sidecar

In [ ]:
set_default_sidecar("CVW 1")

In [ ]:
name = "edges"
cv = show(
    objects[name], 
    states[name],
    height=600,
    cad_width=800,
    reset_camera=True,
    js_debug=True
)

In [ ]:
name = "faces"
cv2 = show(
    objects[name], 
    states[name],
    height=600,
    cad_width=800,    
    title="CVW 2"
)

In [ ]:
name = "vertices"
cv2 = show(
    objects[name], 
    states[name],
    reset_camera=False,
    title="CVW 2"
)

In [ ]:
cv.dump_model()

In [ ]:
get_sidecars()

In [ ]:
get_sidecar("CVW 1") == cv

In [ ]:
get_sidecar("CVW 2") == cv

In [ ]:
close_sidecars()

# Cell Viewer Handling

In [ ]:
name = "hexapod"
cv3 = show(
    objects[name],
    states[name],
    height=600,
    cad_width=800,
    control="trackball",
    tools=True,

    axes=True,
    axes0=True,
    grid=[True, False, True],
    transparent=True,
    black_edges=True,
    ortho=False,
    timeit=True,
    # normal_len=5,
)

In [ ]:
name = "faces"
cv4 = show(
    objects[name],
    states[name],

    cad_width=400,
    height=300,
    glass=True,
    collapse=2,
    pinning=True
)
cv4.remove_ui_elements(["axes", "axes0", "grid", "ortho", "more", "help"])

# Camera location handling

## Trackball controls

In [ ]:
name = "edges"

cv1 = show(
    objects[name], 
    states[name],
    height=600,
    cad_width=800,
    title="Trackball",
    reset_camera=True,
    js_debug=True
)

In [ ]:
cv1.position=(96.5764, -1.7474, 37.7064)
cv1.quaternion=(0.4059, 0.3049, 0.7413, 0.4389)
cv1.zoom=0.6
cv1.target=(6.9493, -11.6226, -12.2272)

**Do not reset camera location**

In [ ]:
name = "faces"
show(
    objects[name], 
    states[name],
    title="Trackball",
    reset_camera=False
)

**Reset camera location**

In [ ]:
name = "faces"
cv = show(
    objects[name], 
    states[name],
    title="Trackball",
    reset_camera=True
)

## Orbit controls

In [ ]:
cv = open_viewer(
    title="Orbit",
    cad_width=700,
    height=525,
)

**Setting camera location during show will also set the reset location of the camera**

In [ ]:
name = "edges"

show(
    objects[name], 
    states[name],
    title="Orbit",
    control="orbit",
    position=(-43.3, 73.7, -39.3),
    zoom=0.5,
    reset_camera=True
)
cv.position, cv.quaternion, cv.target, cv.zoom

In [ ]:
cv.position = (85, 25, 55)
cv.target = (0,0,0)
cv.zoom = 0.8
cv.position, cv.quaternion, cv.zoom

In [ ]:
cv.position, cv.quaternion, cv.target, cv.zoom

**Quaternions with orbit control can be accessed from widget, however, for information only**

In [ ]:
cv.widget.quaternion

# Property access

In [ ]:
cv = open_viewer(
    title = "Examples",
    anchor="right",
    cad_width=700,
    height=525,
    glass=True
)

In [ ]:
menu = widgets.Dropdown(
    options=names,
    value=names[0],
    description='Number:',
    disabled=False,
)

control = "trackball"

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        name = change['new']
        show(    
            objects[name],
            states[name],
            title="Examples",
            control=control,
            js_debug=True
        )

menu.observe(on_change)

show(
    objects[names[0]],
    states[names[0]],
    title="Examples",
    control=control,
#    zoom=0.75,
    js_debug=True
)
menu

## Widget interaction

In [ ]:
cv.update_states({
    '/bottom/bottom_0': (1,0),
    '/bottom/top/top_0': [0,1],
})

In [ ]:
cv.update_states({
    '/bottom/bottom_0': (1,1),
    '/bottom/top/top_0': [1,1],
})

In [ ]:
cv.widget.collapse = 2

In [ ]:
cv.widget.collapse = 1

In [ ]:
cv.ambient_intensity = 0.9
cv.direct_intensity = 0.5

In [ ]:
cv.ambient_intensity = 0.5
cv.direct_intensity = 0.3

In [ ]:
ec = cv.default_edge_color

In [ ]:
cv.default_edge_color = "#ff0000"

In [ ]:
cv.default_edge_color = ec

In [ ]:
cv.grid = [not g for g in cv.widget.grid]

In [ ]:
cv.axes = not cv.axes
cv.axes0 = not cv.axes0
cv.transparent = not cv.transparent
cv.black_edges = not cv.black_edges

In [ ]:
cv.tools = not cv.tools

In [ ]:
cv.ortho = not cv.ortho

In [ ]:
cv.grid = [not g for g in cv.widget.grid]
cv.axes = not cv.axes
cv.axes0 = not cv.axes0
cv.transparent = not cv.transparent
cv.black_edges = not cv.black_edges
cv.tools = not cv.tools
cv.glass = not cv.glass
cv.ortho = not cv.ortho

In [ ]:
cv.glass = not cv.glass

In [ ]:
cv.zoom_speed = 5
cv.pan_speed = 5
cv.rotate_speed = 5

In [ ]:
cv.zoom_speed =1
cv.pan_speed =1
cv.rotate_speed =1

In [ ]:
cv.last_pick

## Clipping handling

In [ ]:
cv.select_clipping()

In [ ]:
cv.clip_intersection = not cv.clip_intersection

In [ ]:
cv.clip_planes = not cv.clip_planes

In [ ]:
cv.clip_value_0 = 10
cv.clip_value_1 = -50
cv.clip_value_2 = 40

In [ ]:
cv.clip_normal_0

In [ ]:
cv.clip_value_2

In [ ]:
cv.clip_normal_0 = (-0.35, -0.35, -0.35)

In [ ]:
cv.clip_normal_0 = (-1, 0, 0)

In [ ]:
cv.select_tree()

# Rotations
## Trackball Control

In [ ]:
name = "hexapod"
cv = show(
    objects[name], 
    states[name],
    control="trackball",
    title="Examples",
    reset_camera=True,
    glass=False
)

In [ ]:
for i in range(10):
    cv.rotate_x(1)
    cv.rotate_y(3)
    cv.rotate_z(5)
    time.sleep(0.05)

In [ ]:
for i in range(10):
    cv.rotate_z(-5)
    cv.rotate_y(-3)
    cv.rotate_x(-1)
    time.sleep(0.05)    

## Orbit control

In [ ]:
name = "hexapod"
cv = show(
    objects[name], 
    states[name],
    control="orbit",
    title="Examples",
    reset_camera=True
)

In [ ]:
for i in range(10):
    cv.rotate_up(3)
    cv.rotate_left(1)
    time.sleep(0.05)

In [ ]:
for i in range(10):
    cv.rotate_left(-1)
    cv.rotate_up(-3)
    time.sleep(0.05)    

# Animation

In [ ]:
name = "hexapod"
cv = show(
    objects[name],
    states[name],
    title="Animation",
    height=600,
    cad_width=800,
    control="trackball",
    tools=True,

    axes=True,
    axes0=True,
    grid=[True, False, False],
)

In [ ]:
import numpy as np
horizontal_angle = 25

leg_names = {
    "right_back", "right_middle", "right_front", 
    "left_back",  "left_middle", "left_front",
    
}

def intervals(count):
    r = [ min(180, (90 + i*(360 // count)) % 360) for i in range(count)]
    return r 

def times(end, count):
    return np.linspace(0, end, count+1).tolist()
    
def vertical(count, end, offset, reverse):
    ints = intervals(count)
    heights = [round(35 * np.sin(np.deg2rad(x)) - 15, 1) for x in ints]
    heights.append(heights[0])
    return times(end, count), heights[offset:] + heights[1:offset+1]

def horizontal(end, reverse):
    factor = 1 if reverse else -1
    return times(end, 4), [0, factor * horizontal_angle, 0, -factor * horizontal_angle, 0]

leg_group = ("left_front", "right_middle", "left_back")

In [ ]:
tracks = []

for name in leg_names:
    # move upper leg
    cv.add_track(AnimationTrack(f"/bottom/{name}", "rz", *horizontal(4, "middle" in name)))

cv.animate(3)
cv.play()

In [ ]:
cv.stop()

In [ ]:
for name in leg_names:
    # move lower leg
    cv.add_track(AnimationTrack(f"/bottom/{name}/lower", "rz", *vertical(8, 4, 0 if name in leg_group else 4, "left" in name)))
    
cv.animate(2)
cv.play()

In [ ]:
cv.clear_tracks()

In [ ]:
close_sidecars()